# Helper functions

In [10]:
from IPython.display import Image
import os
import shutil
from os import listdir
from os.path import isfile, join

In [11]:
def create_empty_dir(dir):
    """
    Creates a directory.
    If the fodler exists, it clears it's content by recreating it.
    @param: dir - folder's path string
    """
    if os.path.isdir(dir):
        shutil.rmtree(dir)    
    os.mkdir(dir)

In [12]:
def get_file_list(dir, ext=None):
    """
    Returns the file list of the given folder.

    @param dir - folder's path string
    @param ext - extensions filter list. It could be str or a list
    @return list of files in the folder
    """
    # single extension
    if type(ext) == str:
        return [join(frame_dir, f) for f in listdir(frame_dir) if isfile(join(frame_dir, f)) and f.split(".")[-1] == ext]

    # extension list
    if type(ext) == list:
        return [join(frame_dir, f) for f in listdir(frame_dir) if isfile(join(frame_dir, f)) and f.split(".")[-1] in ext]
    
    # no extensions
    return [join(frame_dir, f) for f in listdir(frame_dir) if isfile(join(frame_dir, f))]

# Darknet

In [18]:
%cd /content/

/content


In [19]:
# based on https://colab.research.google.com/drive/1_GdoqCJWXsChrOiY8sZMr_zbr_fH-0Fg?usp=sharing#scrollTo=GQQrAMdXN22a
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14086, done.
remote: Total 14086 (delta 0), reused 0 (delta 0), pack-reused 14086
Receiving objects: 100% (14086/14086), 12.72 MiB | 6.67 MiB/s, done.
Resolving deltas: 100% (9585/9585), done.


In [20]:
# enable GPU and OPENCV in the makefile
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [21]:
# check CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [22]:
# build darknet library
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
  

# Copy custom files

In [27]:
# manully annotated smaller dataset (50 images)
# !unzip /content/drive/My\ Drive/20200722/task_garbage_detection_2-2020_07_23_13_39_54-yolo-1.1.zip -d /content/darknet/data/

# generated and manually fixed dataset (322 images)
!unzip /content/drive/My\ Drive/20200722/task_garbage_det_fps_15_img_annot_322-2020_07_24_13_19_02-yolo-1.1.zip -d /content/darknet/data/

Archive:  /content/drive/My Drive/20200722/task_garbage_det_fps_15_img_annot_322-2020_07_24_13_19_02-yolo-1.1.zip
 extracting: /content/darknet/data/obj.data  
 extracting: /content/darknet/data/obj.names  
 extracting: /content/darknet/data/train.txt  
 extracting: /content/darknet/data/obj_train_data/fps_15_frame_0227.txt  
 extracting: /content/darknet/data/obj_train_data/fps_15_frame_0317.jpg  
 extracting: /content/darknet/data/obj_train_data/fps_15_frame_0026.jpg  
 extracting: /content/darknet/data/obj_train_data/fps_15_frame_0212.jpg  
 extracting: /content/darknet/data/obj_train_data/fps_15_frame_0036.jpg  
 extracting: /content/darknet/data/obj_train_data/fps_15_frame_0276.txt  
 extracting: /content/darknet/data/obj_train_data/fps_15_frame_0113.txt  
 extracting: /content/darknet/data/obj_train_data/fps_15_frame_0176.txt  
 extracting: /content/darknet/data/obj_train_data/fps_15_frame_0121.jpg  
 extracting: /content/darknet/data/obj_train_data/fps_15_frame_0167.txt  
 extra

In [28]:
!cp /content/drive/My\ Drive/20200722/yolov4-custom.cfg ./cfg/

cp: cannot create regular file './cfg/': Not a directory


In [29]:
objd_path = 'data/obj.data'
cfg_path = 'cfg/yolov4-custom.cfg'

In [30]:
!cat {data_path}
#!cat {cfg_path}

classes = 5
train = data/train.txt
names = data/obj.names
backup = backup/


# Train model on custom dataset

In [ ]:
# download COCO dataset
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
weight_path = 'yolov4.conv.137'

In [ ]:
!./darknet detector train {objd_path} {cfg_path} {weight_path} -dont_show -map

In [ ]:
Image('chart.png')

## Store weights on drive

In [ ]:
from distutils.dir_util import copy_tree

In [ ]:
backup_path = '/content/drive/My Drive/20200726/yolo_backup'
orig_backup_dir = '/content/darknet/backup'

create_empty_dir(backup_path)
copy_tree(orig_backup_dir, backup_path)

# Detections

### Import existing model

In [ ]:
weight_path = '/content/drive/My\ Drive/20200724/yolo_backup/yolov4-custom_best.weights'

## Run detection on a single image

In [ ]:
img_path = '/content/darknet/data/obj_train_data/fps_15_frame_0034.jpg'

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-custom.cfg {weight_path} {img_path} \
        -thresh 0.1 -dont_show

Image('predictions.jpg')

## Run detection on a video

In [ ]:
video_path = '/content/drive/My\ Drive/20200724/VID_20200722_115436_stabiilizo_annot.mp4'
video_out = '/content/results.avi'

In [ ]:
!./darknet detector demo {data_path} {cfg_path} {weight_path} \
    -dont_show {video_path} -i 0 -out_filename {video_out} -thresh 0.1

In [ ]:
!cp ./results.avi /content/drive/My\ Drive/20200722/

# Generate annotations

## Convert Video to frames and create frame_list.txt 

In [ ]:
# Video to frames
frame_dir = '/content/frames'
input_video = '/content/drive/My\ Drive/20200724/VID_20200722_115436_stabiilizo_annot.mp4'
create_empty_dir(frame_dir)

fps = 15
frames_name = os.path.join(frame_dir, "fps_{}_frame_%04d.jpg".format(fps))
!ffmpeg -i {input_video} -vf fps={fps} -qscale:v 2 {frames_name}

In [ ]:
frame_dir = '/content/frames'
frame_list_file = '/content/frames_list.txt'

frame_list = get_file_list(frame_list_file, ext='jpg')
print("Number of frames: {}".format(len(frame_list)))

# write frame list to file
with open(frame_list_file, 'w') as f:
    for frame_path in frame_list:
        f.write(frame_path + "\n")

## Generate json with the trained model

In [ ]:
weight_path = '/content/drive/My\ Drive/20200724/yolo_backup/yolov4-custom_best.weights'
output_json_path = 'content/result.json'

In [ ]:
!./darknet detector test {data_path} {cfg_path} {weight_path} -dont_show -out /content/result.json -ext_output \
    <{frame_list_file}> /content/result.txt -thresh 0.1

## Convert JSON to Pascal VOC

In [ ]:
!pip install pascal-voc-writer

In [ ]:
import json
from pprint import pprint
from PIL import Image
from pascal_voc_writer import Writer

In [ ]:
with open('/content/result.json') as f:
    data = json.load(f)

In [ ]:
# clear folder if exsists
annot_dir = '/content/pascal_voc'
create_empty_dir(annot_dir)

for det_result in data:
    # pprint(image_res)
    img_path = det_result['filename']
    img = Image.open(img_path)
    width, height = img.size

    # Writer(path, width, height)
    writer = Writer(img_path, width, height)

    for obj in det_result['objects']:
        bb_x_center = obj['relative_coordinates']['center_x']
        bb_y_center = obj['relative_coordinates']['center_y']
        
        bb_width = obj['relative_coordinates']['width']
        bb_height = obj['relative_coordinates']['height']

        xmin = int((bb_x_center - bb_width/2) * width)
        xmax = int((bb_x_center + bb_width/2) * width)
        
        ymin = int((bb_y_center - bb_height/2) * height)
        ymax = int((bb_y_center + bb_height/2) * height)

        # ::addObject(name, xmin, ymin, xmax, ymax)
        writer.addObject(obj['name'], xmin, ymin, xmax, ymax)

    #image name without extension
    xml_name = img_path.split("/")[-1].split('.')[0] + ".xml"
    xml_path = os.path.join(annot_dir, xml_name)
    writer.save(xml_path)
    # print(xml_name)

In [ ]:
!cd /content/; zip frames/pascal_voc.zip pascal_voc/*
!cd /content/; zip garbage_det_fps_15_img_annot_322.zip frames/*

In [ ]:
!cp /content/garbage_det_fps_15_img_annot_322.zip /content/drive/My\ Drive/20200722

# Tensorflow -Yolov4

In [1]:
!cp /content/drive/My\ Drive/20200724/VID_20200722_115436_stabiilizo_annot.mp4 /content/

In [2]:
data_path = '/content/darknet/data/obj.data'
cfg_path = '/content/darknet/cfg/yolov4-custom.cfg'
weight_path = '/content/drive/My\ Drive/20200724/yolo_backup/yolov4-custom_best.weights'
img_path = '/content/fps_5_frame_0002.jpg'
input_video = '/content/VID_20200724_115436_stabiilizo_annot.mp4'

In [ ]:
# !git clone https://github.com/hunglc007/tensorflow-yolov4-tflite
!git clone https://github.com/bessszilard/tensorflow-yolov4-tflite

In [ ]:
%cd /content/tensorflow-yolov4-tflite/
!git checkout add_video_output_and_dont_show_flag #add_custom_name_flag_to_save_model 

In [ ]:
!cd /content/tensorflow-yolov4-tflite/; pip install -r requirements.txt

In [23]:
config_path = '/content/tensorflow-yolov4-tflite/core/config.py'
custom_name_path = "/content/darknet/data/obj.names"

# Read in the file
with open(config_path, 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace("./data/classes/coco.names", custom_name_path)

# Write the file out again
with open(config_path, 'w') as file:
  file.write(filedata)


In [ ]:
%cd /content/tensorflow-yolov4-tflite
create_empty_dir('/content/yolov4-416/')
!python save_model.py --weights {weight_path} --output /content/yolov4-416 --input_size 416 --model yolov4

In [32]:
tf_weights = '/content/yolov4-416'

In [ ]:
!python detectvideo.py --weights /content/yolov4-416 --size 416 --model yolov4 \
    --video /content/VID_20200722_115436_stabiilizo_annot.mp4 \
    --output /content/results.avi \
    --dis_cv2_window